# Environment Configuration

In [ ]:
!pip install torch
!pip install torchvision
!pip install plotly
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 800.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 581.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 658.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 1.7 MB/s eta 0:00:00


In [ ]:
from torchvision import datasets
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR, ExponentialLR
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np
# Data Visualization
import plotly.express as px
from tqdm.auto import tqdm
# Experimental
from torchvision import models

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Data and model will be loaded on the {device}')

Data and model will be loaded on the cpu


# Colon Dataset

In [ ]:
s_1 = 0.2
colon_dataset =  datasets.ImageFolder(
    '/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon',
    transform = transforms.Compose([
        # Downscale the image by s_1
        transforms.Resize(int(s_1 * 500)),
        transforms.ColorJitter(),
        transforms.ToTensor()
    ])
    )

In [ ]:
loaders = {
    'train' : DataLoader(
    colon_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=1
    )
}

In [ ]:
print(colon_dataset.imgs)
data, target = colon_dataset[0]
print(f'Label: {target}')
px.imshow(np.transpose(data, (1, 2, 0)))

[('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon/Negative/img1.bmp', 0), ('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon/Negative/img100.bmp', 0), ('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon/Negative/img11.bmp', 0), ('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon/Negative/img12.bmp', 0), ('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon/Negative/img15.bmp', 0), ('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon/Negative/img16.bmp', 0), ('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon/Negative/img2.bmp', 0), ('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6322/Course Project/CustomData/Colon/Negative/img23.bmp', 0), ('/content/drive/MyDrive/Graduat

# $a(\theta)$: Attention Map for Down-scaled Input Image

In [ ]:
"""
This is the implementation of a(\theta) defined by the paper appendix with some modifications due to
a lack of details...
"""
class step_1_attention_lenet(nn.Module):
  def __init__(self):
    # Superconstructor to inherit nn.Module Properties
    super(step_1_attention_lenet, self).__init__()
    # Layers
    self.conv1 = nn.Conv2d(3, 8, 3, 1, 1)
    self.conv2 = nn.Conv2d(8, 8, 3, 1, 1)
    self.conv3 = nn.Conv2d(8, 1, 3, 1, 1)
    self.average_pooling = nn.AvgPool2d(1)
  def forward(self, x):
    x = F.tanh(self.conv1(x))
    x = F.tanh(self.conv2(x))
    x = F.tanh(self.conv3(x))
    x = self.average_pooling(x)
    x = F.softmax(x, dim = 2)
    return x

# $b(\theta)$: Attention Map for Attention Map

In [ ]:
"""
This is the implementation of b(\theta) defined by the paper appendix with some modifications due to
a lack of details...
"""
class step_2_attention_lenet(nn.Module):
  def __init__(self):
    # Superconstructor to inherit nn.Module Properties
    super(step_2_attention_lenet, self).__init__()
    self.conv1 = nn.Conv2d(3, 8, 3, 1, 1)
    self.conv2 = nn.Conv2d(8, 8, 3, 1, 1)
    self.conv3 = nn.Conv2d(8, 1, 3, 1, 1)
  def forward(self, x):
    x = F.tanh(self.conv1(x))
    x = F.tanh(self.conv2(x))
    x = F.softmax(self.conv3(x), dim = 2)
    return x

## $f(\theta):$ Feature Extractor

In [ ]:
class step_2_feature_extraction(nn.Module):
  def __init__(self):
    # Superconstructor to inherit nn.Module Properties
    super(step_2_feature_extraction, self).__init__()
    self.conv1 = nn.Conv2d(32, 32, 7, 1, 3)
    self.conv2 = nn.Conv2d(32, 32, 3, 1, 1)
    self.conv3 = nn.Conv2d(32, 32, 3, 1, 1)
    self.average_pooling = nn.AvgPool2d(1)
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.relu(self.conv3(x))
    x = self.average_pooling(x)
    return x

## Classification Module $g(\theta)$

In [ ]:
class step_2_classification(nn.Module):
  def __init__(self):
    # Superconstructor to inherit nn.Modules
    super(step_2_classification, self).__init__()
    self.fc = nn.Linear(32, 2)
  def forward(self, x):
    return self.fc(x)

# Zoom-In Network

In [ ]:
class AttentionNetwork(nn.Module):
    def __init__(self):
        super(AttentionNetwork, self).__init__()
        # Input has images of dimension (125x 125)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 100, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(100 * 25 * 25, 512)
        self.fc2 = nn.Linear(512, 2)  # 2 for binary classification

        # Attention Network
        self.attention = nn.Sequential(
            nn.Linear(100 * 25 * 25, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        # output_size(x) = (100 - 3 + 2*1) / 1 + 1 = 100
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        # output_size(x) = (100 - 2 + 2*0)/2 + 1 = 50
        x = torch.relu(self.conv2(x))
        # output_size = (50 - 3 + 2 * 1)/1 + 1 = 50
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        # output_size = (50 - 2 + 2*0)/2 + 1 = 25
        x = torch.relu(self.conv3(x))
        # output_size = 25

        # Calculate attention weights
        attention_weights = self.attention(x.view(x.size(0), -1))

        # Apply attention
        x = x.view(x.size(0), -1) * attention_weights

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Initialize your model
model = AttentionNetwork()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model.parameters(),
    betas = (0.9, 0.999),
    lr = 0.001,
    weight_decay=1e-5
    )
scheduler = ExponentialLR(optimizer, gamma=0.9)

# Training loop
epochs = 1

for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0
    correct = 0
    print(f'Epoch: {epoch}')
    for image, label in tqdm(loaders['train']):
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(output, 1)
        correct += (predicted == label).sum().item()
    scheduler.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {round(running_loss/len(loaders['train']), 2)}")
    print(f"Accuracy:\n\tCorrect: {correct}\n\tSamples: {100}\n\tScore: {round((correct / 100), 2) * 100}")